# [ Overview ]

### (1) Objective : 
> #### 20 지역에서 관측한 날씨에 따른, 월마트 매장 45곳에서 판매하는 111가지 품목의 판매량( `units` )을 예측한다.

### (2) Data : 
> #### train : 4617600 rows, 4 columns  
> - 2012년 1월 1일 ~ 2014년 10월 31일 사이 월마트 매장 45곳에서 판매하는 111가지 품목의 판매량 데이터  

> #### test : 526917 rows, 3 columns
> - 2013년 4월 1일 ~ 2014년 10월 26일 사이 월마트 매장 44곳 (35번 매장 누락) 에 대한 템플릿 

> #### key : 45 rows, 2 columns
> - 20 지역의 관측소와, 같은 지역에 위치한 월마트 매장 45곳을 레이블링한 표  

> #### weather : 20517 rows, 20 columns
> - 2012년 1월 1일 ~ 2014년 10월 31일 사이 20지역의 날씨 데이터  
> - 온도 : Fahrenheit
- 풍속 : mph
- `snowfall`, `preciptotal` : inch  
- M : Missing 
- T : Trace ( 땅이 젖을 정도로만 적게 측정 됨 )  

>- **Weather Data 각 Columns의 정의**
    - date : 날짜
    - tmax : 최대 온도
    - tmin : 최저 온도
    - depart : 과거 30년 동안의 최대온도의 평균과 `tmax`값의 차
    - dewpoint : 평균 이슬점
    - wetbulb : 평균 습도
    - heat : 난방 지수 (65 - `avg`) 1 ~ 6월 적용
    - cool : 냉방 지수 (`tavg` - 65) 7 ~ 12월 적용
    - sunrise : 일출 시간
    - sunset : 일몰 시간 
    - codesum : 특이 날씨 코드
        - +FC : TORNADO/WATERSPOUT
        - FC : FUNNEL CLOUD
        - TS : THUNDERSTORM
        - GR : HAIL
        - RA : RAIN
        - DZ : DRIZZLE
        - SN : SNOW
        - SG : SNOW GRAINS
        - GS : SMALL HAIL &/OR SNOW PELLETS
        - PL : ICE PELLETS
        - IC : ICE CRYSTALS
        - FG+: HEAVY FOG (FG & LE.25 MILES VISIBILITY)
        - FG : FOG
        - BR : MIST
        - UP : UNKNOWN PRECIPITATION
        - HZ : HAZE
        - FU : SMOKE
        - VA : VOLCANIC ASH
        - DU : WIDESPREAD DUST
        - DS : DUSTSTORM
        - PO : SAND/DUST WHIRLS
        - SA : SAND
        - SS : SANDSTORM
        - PY : SPRAY
        - SQ : SQUALL
        - DR : LOW DRIFTING
        - SH : SHOWER
        - FZ : FREEZING
        - MI : SHALLOW
        - PR : PARTIAL
        - BC : PATCHES
        - BL : BLOWING
        - VC : VICINITY
        - (-) : LIGHT
        - (+) : HEAVY
        - 표기가 안 되어 있다면 : moderate weather
    - snowfall : 눈이 쌓인 정도 (inches)
    - precipitotal : 24시간 기준 강수량 (inches)
    - stnpressure : 평균 기압
    - sealevel : 해수면 압력
    - resultspeed : 합성 풍속 (mph)
    - resultdir : 합성 풍향 (10도씩) 
    - avgspeed : 평균 풍속 (mph)

        

### (3) : Evaluation
> - 예측값과 실제값의 차이를 Root-Mean-Squared-Logarithmic-Error(RMSLE)로 채점한다.

In [ ]:
from fbprophet import Prophet
from scipy.linalg import toeplitz
from tqdm import tnrange, tqdm_notebook
from time import sleep

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
weather = pd.read_csv('data/weather.csv')
key = pd.read_csv('data/key.csv')

In [ ]:
len(test), len(train), len(weather), len(key)

In [ ]:
train['store_nbr'].value_counts().sort_index()

In [ ]:
# 35번 누락
test['store_nbr'].value_counts().sort_index()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
# 문자열이기에 nan값이 나타나지 않음
weather.isna().sum()

# 데이터 전처리

데이터를 편하게 다루기위해 test와 train을 합친다.

In [ ]:
train['logunits'] = np.log1p(train['units'])
test['logunits']=np.nan
test['units']=np.nan
merged = pd.concat([train, test])
merged = merged.sort_values(by=['date','store_nbr','item_nbr'])
merged['date'] = pd.to_datetime(merged['date']) 

In [ ]:
set(train['item_nbr'].value_counts())

In [ ]:
train['store_nbr'].value_counts()

판매된 아이템만 찾기

In [ ]:
def get_nonzero_item():
    nonzero_res = {}
    for n in range(1, 46):
        s1 = train[train['store_nbr']==n]
        nonzero = s1.pivot_table(index='item_nbr', aggfunc=np.sum).reset_index()
        nonzero = nonzero[nonzero['units']>0]
        
        nonzero_res[n] = nonzero['item_nbr'].values.tolist()
        #print(n, nonzero['item_nbr'].values)
        
    retailed_items = []
    for store_nbr in range(1, 46): 
        items = nonzero_res[store_nbr]
        res = list(zip([store_nbr] * len(items), items))
        retailed_items.extend(res)
    
    return pd.DataFrame(retailed_items, columns=['store_nbr','item_nbr'])

retailed_items = get_nonzero_item()
retailed_items['t']=1
retailed_items

In [ ]:
rdf = retailed_items.pivot_table(index='item_nbr',columns='store_nbr', values='t',aggfunc=np.sum)
plt.figure(figsize=(12, 15))
sns.heatmap(rdf)
plt.yticks(fontsize=8)
plt.xticks(fontsize=8)
plt.show()

In [ ]:
retailed_items.pivot_table(index='item_nbr', values='t',aggfunc=np.sum).sort_values('t', ascending=False)

45개 매장에서 공통으로 많이 팔리는 아이템은 93, 5, 9, 45으로 분리해서 분석할 필요가 있어 보인다

## 날씨 데이터 전처리

In [ ]:
# 불필요한 문자열 변환
weather2 = weather.replace('-', np.nan) 
weather2 = weather2.replace('M', np.nan) 
weather2 = weather2.replace('T', 0)
weather2 = weather2.replace('  T', 0)

# 문자열에서 날짜로 변환
weather2.date = pd.to_datetime(weather2.date)

# 문자열에서 float으로 변환
for col in ['tmax', 'tmin', 'tavg', 'depart', 'dewpoint','wetbulb', 'heat', 'cool', 'resultdir']:
    weather2[col] = weather2[col].astype(float)
            
for col in ['snowfall','preciptotal', 'stnpressure', 'sealevel', 'resultspeed','avgspeed']:
    weather2[col] = weather2[col].astype(float)

### 날씨데이터 누락값 파악

In [ ]:
#null값 파악
weather2.isna().sum()

In [ ]:
# 5번 관측소만 데이터 부족
weather2['station_nbr'].value_counts()

### 날씨데이터 누락값 시계열로 처리하기 - fbprophet 사용

In [ ]:
for col in ['tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed']:
    
    print(col)
    plt.figure(figsize=(20, 10))
    for station_nbr in range(1, 21):
        s1 = weather2[weather2.station_nbr == station_nbr]
        plt.subplot(4, 5, station_nbr)
        plt.plot(s1.date, s1[col])
        plt.xticks([])
        plt.title('station {}'.format(station_nbr))
    plt.tight_layout(pad=1, h_pad=1)
    plt.show()
    

날씨 데이터의 누락값은 시계열분석으로 처리가능므로 arima 모델을 사용하여 처리한다
5번 관측소는 데이터는 면밀히 살펴볼 필요가 있다

In [ ]:
# 시계열 모델 만들기

for col in tqdm_notebook(['tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed'], desc='col'):
    
    naidx_ls = []
    predval_ls = []

    for station_nbr in range(1, 21):
        
        # 5번 제외하고 시계열 에측 시작
        if station_nbr == 5:
            continue
            
        try:
            s1 = weather2[weather2['station_nbr'] == station_nbr]

            df = s1.filter(['date',col]).reset_index().rename(columns={'date':'ds',col:'y'})
            m = Prophet(daily_seasonality=True, 
                        yearly_seasonality=True, 
                        weekly_seasonality=True, 
                        growth='linear' )
            m.fit(df)

            future = m.make_future_dataframe(periods=0) 
            forecast = m.predict(future)

            yhat = forecast.yhat
            naidx = df.y[df.y.isna()].index

            dfnaidx = df[df.y.isna()]['index'].values.tolist()
            naidx_ls.extend(dfnaidx)

            predval_ls.extend(yhat[naidx])
            #df.y[naidx] = yhat[naidx]
            #plt.figure(figsize=(15, 3))
            #plt.plot(df.y, label='y')
            #plt.plot(forecast.yhat, label='yhat')
            #plt.legend()
            #plt.show()
        except Exception as e:
            print('error %s - station %d'%(col, station_nbr))
            
    weather2[col].loc[naidx_ls] = predval_ls
    del predval_ls
    del naidx_ls

In [ ]:
# 누락값 제거 확인
weather2.isna().sum()

In [ ]:
for col in tqdm_notebook(['tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed']):
    
    print(col)
    plt.figure(figsize=(20, 10))
    for station_nbr in range(1, 21):
        s1 = weather2[weather2.station_nbr == station_nbr]
        plt.subplot(4, 5, station_nbr)
        plt.plot(s1.date, s1[col])
        plt.xticks([])
        plt.title('station {}'.format(station_nbr))
    plt.tight_layout(pad=1, h_pad=1)
    plt.show()
    

In [ ]:
weather3 = weather2.copy()

In [ ]:
# 문제있는 5, 8번 제외하고 누락데이터 추정
s5 = weather3[weather3['station_nbr'] == 5]
s5['index'] = s5.index
# station 8 sealevel 채우기
s8 = weather3[weather3['station_nbr'] == 8]
s8['index'] = s8.index
sn5 = weather3[(weather3['station_nbr'] != 5) & (weather3['station_nbr'] != 8)]

In [ ]:
# 5번 스테이션 tmax는 다른 스테이션의 평균 값으로 대체
avgtmp = sn5.pivot_table(index='date', aggfunc=np.average).reset_index()
s5 = s5.merge(avgtmp, how='left', on='date')

In [ ]:
def checknan(a, b):
    res = a
    if np.isnan(a):
        res = b
    else:
        res = a
    return res

In [ ]:
s5['tmax'] = s5.apply(lambda row: checknan(row['tmax_x'], row['tmax_y']), axis=1)
s5['tmin'] = s5.apply(lambda row: checknan(row['tmin_x'], row['tmin_y']), axis=1)
s5['tavg'] = s5.apply(lambda row: checknan(row['tavg_x'], row['tavg_y']), axis=1)
s5['dewpoint'] = s5.apply(lambda row: checknan(row['dewpoint_x'], row['dewpoint_y']), axis=1)
s5['wetbulb'] = s5.apply(lambda row: checknan(row['wetbulb_x'], row['wetbulb_y']), axis=1)
s5['heat'] = s5.apply(lambda row: checknan(row['heat_x'], row['heat_y']), axis=1)
s5['cool'] = s5.apply(lambda row: checknan(row['cool_x'], row['cool_y']), axis=1) 
s5['stnpressure'] = s5.apply(lambda row: checknan(row['stnpressure_x'], row['stnpressure_y']), axis=1)
s5['resultspeed'] = s5.apply(lambda row: checknan(row['resultspeed_x'], row['resultspeed_y']), axis=1) 
s5['resultdir'] = s5.apply(lambda row: checknan(row['resultdir_x'], row['resultdir_y']), axis=1)
s5['avgspeed'] = s5.apply(lambda row: checknan(row['avgspeed_x'], row['avgspeed_y']), axis=1)
s5['sealevel'] = s5.apply(lambda row: checknan(row['sealevel_x'], row['sealevel_y']), axis=1)

In [ ]:
s5 = s5.filter(items=['index','date','tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb',
       'heat', 'cool', 'stnpressure', 'resultspeed', 'resultdir', 'avgspeed',
       'sealevel'])

In [ ]:
for col in ['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb',
       'heat', 'cool', 'stnpressure', 'resultspeed', 'resultdir', 'avgspeed',
       'sealevel']:
    weather3[col].loc[s5['index']] = s5[col].values

In [ ]:
for col in tqdm_notebook(['tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed']):
    
    print(col)
    plt.figure(figsize=(20, 10))
    for station_nbr in range(1, 21):
        s1 = weather3[weather3.station_nbr == station_nbr]
        plt.subplot(4, 5, station_nbr)
        plt.plot(s1.date, s1[col])
        plt.xticks([])
        plt.title('station {}'.format(station_nbr))
    plt.tight_layout(pad=1, h_pad=1)
    plt.show()
    

In [ ]:
# sealevel(8)

In [ ]:
weather3['sealevel'].loc[s8.index] = avgtmp['sealevel'].values

In [ ]:
# depart, snowfall, preciptotal,

In [ ]:
depart_from = weather3[weather3['station_nbr'].isin([2, 3, 4, 6, 11, 14, 15, 18, 19])]
depart_from = depart_from.pivot_table(index='date', aggfunc=np.average).reset_index()

In [ ]:
for col in ['depart', 'snowfall', 'preciptotal']:
    for station_nbr in [1, 5, 7, 8, 9, 10, 12, 13, 16, 17, 20]:
        sdf = weather3[weather3['station_nbr'] == station_nbr]
        sdf.insert(0, 'index', sdf.index)
        sdf = sdf.merge(depart_from, how='left', on='date')

        sdf[col] = sdf.apply(lambda row: checknan(row['{}_x'.format(col)], row['{}_y'.format(col)]), axis=1)

        weather3[col].loc[sdf['index']] = sdf[col].values

In [ ]:
for col in tqdm_notebook(['depart', 'snowfall', 'preciptotal','sealevel']):
    
    print(col)
    plt.figure(figsize=(20, 10))
    for station_nbr in range(1, 21):
        s1 = weather3[weather3.station_nbr == station_nbr]
        plt.subplot(4, 5, station_nbr)
        plt.plot(s1.date, s1[col])
        plt.xticks([])
        plt.title('station {}'.format(station_nbr))
    plt.tight_layout(pad=1, h_pad=1)
    plt.show()
    

In [ ]:
# sunrise, sunset, codesum 을 제외한 모두 nan값 처리
weather3.info()

In [ ]:
weather3

In [ ]:
# 날짜 관련 데이터 추가
weather3['date'] = pd.to_datetime(weather3['date'])
weather3['year'] = weather3['date'].dt.year
weather3['month'] = weather3['date'].dt.month
weather3['day'] = weather3['date'].dt.day
weather3['week'] = weather3['date'].dt.week
weather3['weekday'] = weather3['date'].dt.weekday
weather3['quarter'] = weather3['date'].dt.quarter

In [ ]:
# 휴일데이터 크롤링
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def get_holidays(year):
    response = requests.get("https://www.timeanddate.com/calendar/custom.html?year="+str(year)+"&country=1&cols=3&df=1&hol=25")
    dom = BeautifulSoup(response.content, "html.parser")

    trs = dom.select("table.cht.lpad tr")

    df = pd.DataFrame(columns=["ds", "holiday", "lower_window",'upper_window'])
    for tr in trs:
        datestr = tr.select_one("td:nth-of-type(1)").text
        date = datetime.strptime("{} {}".format(year, datestr), '%Y %b %d')
        holiday = tr.select_one("td:nth-of-type(2)").text
        df.loc[len(df)] = {"ds" : date, "holiday": holiday, "lower_window": 0,'upper_window':1}
    return df

def get_blackfriday(): 
    rows=[]
    rows.append([datetime(2012, 11, 23, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2012, 11, 24, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2012, 11, 25, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2012, 11, 26, 0, 0), 'blackfriday', 0, 1])
    
    rows.append([datetime(2013, 11, 29, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2013, 11, 30, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2013, 12, 1, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2013, 12, 2, 0, 0), 'blackfriday', 0, 1])
    
    rows.append([datetime(2014, 11, 28, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2014, 11, 29, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2014, 11, 30, 0, 0), 'blackfriday', 0, 1])
    rows.append([datetime(2014, 12, 1, 0, 0), 'blackfriday', 0, 1])
    
    return pd.DataFrame(rows, columns=['ds', 'blackfriday', 'lower_window', 'upper_window'])

holiday_ls = []
for year in range(2012, 2015):
    df = get_holidays(year)
    holiday_ls.append(df)
holiday_df = pd.concat(holiday_ls)
blackfriday_df = get_blackfriday()
     
#holiday_df = holiday_df.sort_values(by="ds").reset_index(drop=True)
#holiday_df["date"] = holiday_df["date"].astype(str)
#holiday_df["isbf"] = holiday_df["blackfriday"].apply(lambda x: 0 if x==0 else 1)
#holiday_df.to_csv("hb.csv", index=False)

In [ ]:
hb_df = pd.merge(holiday_df, blackfriday_df, how='outer', on=['ds','lower_window','upper_window'])

In [ ]:
weather3 = weather3.merge(hb_df.drop(columns=['lower_window','upper_window']), how='left', left_on='date', right_on='ds')

In [ ]:
weather3 = weather3.drop(columns=['ds'])

In [ ]:
weather3.to_csv('data/weather_m.csv', index=False)

In [ ]:
# weather, train 합치기

In [ ]:
weather3 = pd.read_csv('data/weather_m.csv')
weather3['date'] = pd.to_datetime(weather3['date'])

#merged = merged.drop(columns=['year','month','day','week'])
merged = merged.merge(key, how='left', on='store_nbr')
data = merged.merge(weather3, how='left', on=['date', 'station_nbr'])

분석을 위한 데이터프레임 완성
- 날씨, nan값처리 완료
- 날짜
- 휴일
- 블랙프라이데이

In [ ]:
df1_9 = data[(data['store_nbr']==1) & (data['item_nbr']==9)]
df1_9['isholiday'] = df1_9['holiday'].apply(lambda x: 0 if str(x) == 'nan' else 1)
df1_9['isblackfriday'] = df1_9['blackfriday'].apply(lambda x: 0 if str(x) == 'nan' else 1)

In [ ]:
col_cate = ['year', 'month', 'day', 'week', 'weekday', 'quarter', 'isholiday', 'isblackfriday']
col_num = ['tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool','snowfall', 'preciptotal','stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']

In [ ]:
plt.figure(figsize=(15, 5))
for idx, col in enumerate(col_num):
    plt.subplot(3, 5, idx+1)
    sns.regplot(df1_9[col], df1_9['logunits'])
    plt.grid()
    plt.yticks([])
plt.tight_layout(pad=1, h_pad=1)
plt.show()

날씨에 관해서 판매량과 관련있는 데이터는 찾기 힘들다.
snowfall, preciptotal정도?

In [ ]:
plt.figure(figsize=(15, 2))
for idx, col in enumerate(col_cate):
    try:
        plt.subplot(1,8, idx+1)
        sns.regplot(df1_9[col], df1_9['logunits'])
        plt.grid()
        plt.yticks([])
    except Exception as e:
        print(col)
plt.tight_layout(pad=1, h_pad=1)
plt.show()

날짜에 관련해서 연도, 요일이 이 그래도 영향을 끼친다

In [ ]:
plt.figure(figsize=(15, 3))
plt.subplot(131)
sns.barplot(x='weekday', y='units', data=df1_9)
plt.subplot(132)
sns.boxplot(x='weekday', y='units', data=df1_9)
plt.subplot(133)
sns.boxplot(x='weekday', y='logunits', data=df1_9)
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
plt.subplot(131);sns.barplot(x='year', y='units', data=df1_9) 
plt.subplot(132);sns.boxplot(x='year', y='units', data=df1_9) 
plt.subplot(133);sns.boxplot(x='year', y='logunits', data=df1_9) 
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
y1 = df1_9['logunits'].values[1:]
y2 = df1_9['logunits'].values[:-1]
plt.subplot(1,4,1)
sns.regplot(y1, y2) ; plt.grid(); plt.title(1)

y1 = df1_9['logunits'].values[2:]
y2 = df1_9['logunits'].values[:-2]
plt.subplot(1,4,2)
sns.regplot(y1, y2)  ; plt.grid(); plt.title(2)

y1 = df1_9['logunits'].values[7:]
y2 = df1_9['logunits'].values[:-7]
plt.subplot(1,4,3)
sns.regplot(y1, y2) ; plt.grid(); plt.title(7)

y1 = df1_9['logunits'].values[14:]
y2 = df1_9['logunits'].values[:-14]
plt.subplot(1,4,4)
sns.regplot(y1, y2) ; plt.grid(); plt.title(14)

plt.tight_layout(pad=1, h_pad=1)
plt.show()

1일, 2일, 7일, 14일 상관관계는 다른 날씨 변수보다 높게 나타난다

In [ ]:
x_col = ['tmax', 'tmin', 'tavg',
'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall',
'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
'avgspeed']

In [ ]:
dfX = df1_9[df1_9['units'].notna()].filter(x_col).reset_index(drop=True)
dfy = df1_9[df1_9['units'].notna()].filter(items=['units']).reset_index(drop=True)

In [ ]:
df1_9.to_csv('data/sample_df.csv',index=False)

In [ ]:
# 전체적인 추세 파악하기
# 계절성이 나타나지는 않는다

In [ ]:
def get_store(n):
    return merged[merged['store_nbr']==n]

In [ ]:
def get_store_item(n, item, isview=True):
    s1 = get_store(n)
    i9 = s1[s1['item_nbr']==item].reset_index(drop=True)
    
    i9['istest'] = 0
    i9['istest'][i9['units'].isna()] = i9['units'].max()

    if isview:
        plt.figure(figsize=(20, 1))
        plt.plot(i9['date'], i9['istest'])
        plt.plot(i9['date'], i9['units'], c='r')
        plt.grid()
        plt.title('store : %d - item : %d'%(n, item))
        plt.show()

#         plt.figure(figsize=(15, 5))
#         for idx, year in enumerate([2012, 2013, 2014]):
#             i92012 = i9[i9['year']==year]
#             x = i92012['date']
#             y = i92012['istest'] 
#             plt.subplot(3,1,idx+1)
#             plt.plot(x, y)
#             plt.plot(x, i92012['units'])
#             plt.grid()
#         plt.tight_layout(h_pad=1)
#         plt.show()
    return i9

In [ ]:
for i in range(len(retailed_items)):
    row = retailed_items.loc[i]
    n = row['store_nbr']
    item = row['item_nbr']
    get_store_item(n, item)

대부분 추세를 갖고 있지만 계절성을 보이는 자료도 있다

In [ ]:
# 월별 데이터 
col = 'units'
for month in range(1, 13):
    month12 = i9[(i9['month']==month) & (i9['year']==2012)]
    month13 = i9[(i9['month']==month) & (i9['year']==2013)]
    month14 = i9[(i9['month']==month) & (i9['year']==2014)]
    plt.figure(figsize=(15, 2))
    plt.plot(month12['day'], month12[col])
    plt.plot(month13['day'], month13[col])
    plt.plot(month14['day'], month14[col])
    plt.xticks([])
    plt.title(month)
    plt.show()

In [ ]:
for week in range(1, 53):
    week1 = i9[(i9['week']==week) & (i9['year']==2012)]

    #plt.figure(figsize=(15, 1))
    plt.plot(list(range(len(week1))), week1['units']) 
    #plt.show()